# Script para leer los fichero txt de calidad del aire y añadir el dato en el csv unificado.

Lista de los ficheros a leer: Los ficheros son de los años indicados en el bucle (de 2010 a 2016, incluidos). 2017 y posteriores tienen otro formato.

Lee cada línea del fichero y le inserta ',' después de cada 'V' y no escribe la 'V'. Luego a las posiciones fijas de los datos iniciales también se inserta ','

In [58]:
cabecera = "ESTACION,MAGNITUD,TECNICA,DIARIO,AÑO,MES"
for i in range (1,32)  :
    cabecera = cabecera + ',D' + str(i) + ',V' + str(i)

In [59]:
funificado = '../dataCalidad/datosCalidadUnificado.csv'
f = open (funificado,'w')
f.write(cabecera + '\n')
f.close()

In [60]:
#para incluir de 2010 a 2017 el bucle es del rango [10:17]
for i in range (10,17)  :  
    fichero = '../dataCalidad/datos' + str(i) + '.txt'
    with open(fichero, mode='r') as fentrada:
        for linea in fentrada:
            lineaFinal =''
            setV = False
            for caracter in linea:
                if (caracter == 'V'):
                    lineaFinal = lineaFinal + "," + caracter
                    setV = True
                else:
                    if (setV and caracter != '\n'):
                        lineaFinal = lineaFinal + "," + caracter
                        setV = False
                    else:
                        lineaFinal = lineaFinal + caracter
            
            lineaFinal = lineaFinal[:8] + ',' + lineaFinal[8:10] + ',' + lineaFinal[10:12] + ',' + lineaFinal[12:14] + \
            ',' + lineaFinal[14:16] + ',' + lineaFinal[16:18] + ',' + lineaFinal[20:]
            
            #print(linea)
            #print(lineaFinal)
            #escribe la linea en el fichero de salida 
            with open(funificado, mode='a') as fsalida:
                fsalida.write(lineaFinal)
            


In [49]:
fichero = '../dataCalidad/datos17.csv'
with open(fichero, mode='r') as f:
    first_line = f.readline()
    second_line = f.readline()
    print(first_line)
    print(second_line)

PROVINCIA;MUNICIPIO;ESTACION;MAGNITUD;PUNTO_MUESTREO;ANO;MES;D01;V01;D02;V02;D03;V03;D04;V04;D05;V05;D06;V06;D07;V07;D08;V08;D09;V09;D10;V10;D11;V11;D12;V12;D13;V13;D14;V14;D15;V15;D16;V16;D17;V17;D18;V18;D19;V19;D20;V20;D21;V21;D22;V22;D23;V23;D24;V24;D25;V25;D26;V26;D27;V27;D28;V28;D29;V29;D30;V30;D31;V31

28;079;4;1;28079004_1_38;2017;01;00006;V;00008;V;00012;V;00009;V;00007;V;00010;V;00010;V;00011;V;00010;V;00012;V;00011;V;00008;V;00006;V;00007;V;00007;V;00009;V;00006;V;00005;V;00005;V;00005;V;00006;V;00007;V;00009;V;00012;V;00009;V;00009;V;00005;V;00004;V;00004;V;00005;V;00007;V

